In [ ]:
# =========================================
# 1. Preparación del entorno
# =========================================
!pip install -q transformers accelerate bitsandbytes peft datasets

from google.colab import drive
drive.mount('/content/drive')  # opcional: guardar resultados en tu Drive

In [ ]:
# =========================================
# 2. Imports
# =========================================
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [ ]:
# =========================================
# 3. Modelo base (Mistral 7B Instruct)
# =========================================
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto"
)

# Preparar modelo para fine-tuning LoRA
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj","v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [ ]:
# =========================================
# 4. Dataset Vegeta-like (ejemplo mini)
#    👉 Reemplaza con tu propio archivo JSONL
# =========================================
sample_data = [
    {
      "instruction": "Usuario: ¿Cómo puedo motivarme para entrenar hoy?",
      "response": "¿Motivarte? ¡Tsk! Los débiles buscan excusas. Ponte de pie y demuestra que no eres un insecto."
    },
    {
      "instruction": "Usuario: Dame un consejo para hacer sentadillas",
      "response": "¡Espalda recta, pecho arriba! Si no bajas hasta el paralelo, ni te atrevas a llamarlo entrenamiento."
    },
    {
      "instruction": "Usuario: Estoy cansado, ¿debería parar?",
      "response": "¿Parar? ¡Ridículo! Solo cuando superes tu límite empezarás a ser digno. Haz al menos otra serie."
    }
]

import json
with open("vegeta_dataset.jsonl", "w") as f:
    for row in sample_data:
        f.write(json.dumps(row) + "\n")

dataset = load_dataset("json", data_files="vegeta_dataset.jsonl")

def format_example(example):
    return {"text": f"{example['instruction']}\nVegeta: {example['response']}"}

dataset = dataset.map(format_example)
train_data = dataset["train"]

In [ ]:
=========================================
# 5. Entrenamiento LoRA
# =========================================
training_args = TrainingArguments(
    output_dir="./vegeta-lora",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    tokenizer=tokenizer
)

trainer.train()

# Guardar LoRA en Drive
model.save_pretrained("/content/drive/MyDrive/vegeta-lora")

In [ ]:
# =========================================
# 6. Uso del modelo Vegeta
# =========================================
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True, device_map="auto")
model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/vegeta-lora")
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "Usuario: No quiero hacer más, estoy cansado.\nVegeta:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=80)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))